# <center>**Graded Challenge 3**</center>

* *Name* : Alexander Prasetyo Christianto
* *Age* : 23
* *Last Education Background* : *Electrical Engineering*
* *Occupation* : *Full Time Data Science Student Batch-001*


## Description

### Objective

* Able to get data using BigQuery
* Able to perform data processing before performing calculations and analysis
* Able to apply statistical concepts to problems
* Understanding the basics of Business Knowledge

### Problem Statement

I am a data analyst at The Look which is one of the largest e-commerce platforms on the planet Mars. I am asked to make an evaluation report on sales activity on the platform.

Based on my quick analysis, using SMART problem framing, The Look E-Commerce is losing the market share through January to July 2022 because The Look E-Commerce is not keeping their users in their ecosystem.

## Working Station

### Libraries Used

Here i used pandas and scipy for hypothesis testing later in this assignment. The dataset is fetched and queried directly from the BigQuery Google Public Dataset.

In [ ]:
import pandas as pd
from scipy import stats

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('LETS GOOO!!!')

from google.cloud import bigquery

project_id = "bigquery-public-data-359109"
client = bigquery.Client(project=project_id)

LETS GOOO!!!


---

First of all, i want to know the how many customers are classified as  'High Retention Customer' and 'Low Customer Retention'. Customer will be classified as 'High Retention Customer' when the amount of their order is greater equal to 3 times.

In [ ]:
df = client.query('''
SELECT (SELECT COUNT(CustomerRetentionType) from (SELECT user_id as uid, COUNT(order_id) as oid,
CASE WHEN COUNT(order_id) >= 3 then "High Retention Cust"
  else "Low Retention Cust" end as CustomerRetentionType
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE created_at BETWEEN '2022-01-01 00:00:00 UTC' and '2022-06-01 23:59:59'
GROUP BY uid HAVING CustomerRetentionType = "High Retention Cust"))/(select count(distinct id) from `bigquery-public-data.thelook_ecommerce.users`) * 100 as PercentageOfHighRetCust
''').to_dataframe()

In [ ]:
df

,PercentageOfHighRetCust
0,0.588


The number of retention of The Look users from January to July 2022 is 0.588 % of the total of The Look's users. This figure is still very small, so it can be indicated that there needs to be an evaluation of the products or services owned by The Look.

---

In [ ]:
df2 = client.query('''
SELECT AVG(oid) AS AverageOrderHRC, MIN(oid) AS MinimumOrderHRC, MAX(oid) AS MaximumOrderHRC FROM (SELECT user_id as uid, COUNT(order_id) as oid,
CASE WHEN COUNT(order_id) >= 3 then "High Retention Cust"
  else "Low Retention Cust"
  end as CustomerRetentionType
FROM `bigquery-public-data.thelook_ecommerce.orders`
WHERE created_at BETWEEN '2022-01-01 00:00:00 UTC' and '2022-06-01 23:59:59'
GROUP BY uid HAVING CustomerRetentionType = "High Retention Cust")
''').to_dataframe()

df2

,AverageOrderHRC,MinimumOrderHRC,MaximumOrderHRC
0,3.127551,3,4


For customers who are categorized as customers with high retention, their average purchase value is 3 times. Then, the minimum order frequency is 3 times and the maximum order frequency is 4 times.

---

In [ ]:
df3 = client.query('''
SELECT AVG(Spendings) AS Average_Spending, MIN(Spendings) AS Minimum_Spending, MAX(Spendings) AS Maximum_Spending FROM
     (SELECT odb.user_id AS User, SUM(idb.sale_price) AS Spendings,      
     CASE WHEN COUNT(odb.order_id) >= 3  THEN "High Retention Cust" ELSE "Low Retention Cust" END AS CustomerRetentionType     
     FROM `bigquery-public-data.thelook_ecommerce.orders` as odb INNER JOIN `bigquery-public-data.thelook_ecommerce.order_items` as idb ON odb.order_id=idb.order_id     
     WHERE odb.created_at BETWEEN '2022-01-01 00:00:00 UTC' and '2022-06-01 23:59:59' GROUP BY odb.user_id HAVING CustomerRetentionType = "High Retention Cust")
''').to_dataframe()

df3

,Average_Spending,Minimum_Spending,Maximum_Spending
0,221.520881,24.21,1401.310001


After further analysis, customers who are categorized as customers with retention spend an average of 221.52 USD, the lowest value when placing an order is 24.21 USD and the highest order value is 1401.31 USD.

----

In [ ]:
df4 = client.query('''
SELECT PUD.traffic_source AS Traffic_Source, COUNT(PUD.traffic_source) AS AMT_Traffic_Source FROM (
     SELECT user_id as uid, COUNT(order_id) as oid,
     CASE WHEN COUNT(order_id) >= 3 then "High Retention Cust" 
     else "Low Retention Cust" end as CustomerRetentionType 
     FROM `bigquery-public-data.thelook_ecommerce.orders` 
     WHERE created_at BETWEEN '2022-01-01 00:00:00 UTC' and '2022-06-01 23:59:59' 
     GROUP BY uid HAVING CustomerRetentionType = "High Retention Cust"
) AS CRT
INNER JOIN `bigquery-public-data.thelook_ecommerce.users` AS PUD
ON CRT.uid=PUD.id
GROUP BY PUD.traffic_source
''').to_dataframe()

df4

,Traffic_Source,AMT_Traffic_Source
0,Search,400
1,Email,20
2,Display,34
3,Organic,97
4,Facebook,37


And most of the users who are categorized as users with high retention have traffic sources from their respective search results.

NO. 5

This time I will try to test a case whether the source of traffic has a relationship with customer retention.

From this case, I will test the hypothesis using the chi-square test method.

By using this method, we need to determine the contingency table between traffic sources and customer retention types. And from that case I will define the null hypothesis and the alternative hypothesis.

H0 : traffic source has no relationship with loyal or not a user <br>
H1: traffic source has no relationship with loyal or not a user

In [ ]:
df5 = client.query('''
SELECT PUD.traffic_source as Traffic_Source, 
CRT.CustomerRetentionType as Loyal
FROM(     
     SELECT user_id as uid, COUNT(order_id) as oid,
     CASE WHEN COUNT(order_id) >= 3 then "Ya" 
     else "Tidak" end as CustomerRetentionType 
     FROM `bigquery-public-data.thelook_ecommerce.orders` 
     WHERE created_at BETWEEN '2022-01-01 00:00:00 UTC' and '2022-06-01 23:59:59' 
     GROUP BY uid
) AS CRT
INNER JOIN `bigquery-public-data.thelook_ecommerce.users` AS PUD
ON CRT.uid=PUD.id
''').to_dataframe()

df5

,Traffic_Source,Loyal
0,Email,Tidak
1,Email,Tidak
2,Email,Tidak
3,Email,Tidak
4,Email,Tidak
...,...,...
25143,Facebook,Tidak
25144,Facebook,Tidak
25145,Facebook,Tidak
25146,Facebook,Tidak


In [ ]:
contingency_table=pd.crosstab(df5["Traffic_Source"],df5["Loyal"])
contingency_table

Loyal,Tidak,Ya
Traffic_Source,,
Display,965,34
Email,1204,20
Facebook,1483,37
Organic,3719,97
Search,17189,400


In [ ]:
stat, p, dof, expected = stats.chi2_contingency(contingency_table)
print('p=%.3f' % (p))
if p > 0.05:
    print('Probably independent')
else:
    print('Probably dependent')

p=0.069
Probably independent


Based on the calculation, the result of the P value is 0.069. Because the P value is above 0.05, it causes the null hypothesis to fail to be rejected. So customer retention has nothing to do with traffic sources.

## Analysis Points

- The Look E-Commerce memiliki jumlah user dengan retensi tinggi yang rendah, yaitu hanya sebesar 0.588 %.
- User yang memiliki retensi tinggi, rata-rata mereka melakukan transaksi adalah sebanyak 3 kali dengan rata-rata uang yang dihabiskan sebesar 221.520881 USD. Frekuensi minimal user retensi tinggi melakukan transaksi adalah sebanyak 3 kali dan salah satu user dengan retensi tinggi pernah melakukan pemesanan hanya sebesar 24.21 USD. Frekuensi terbesar user dengan retensi tinggi melakukan transaksi adalah sebanyak 4 kali dan nilai terbesar pemesanan yang pernah dilakukan adalah 1401.310 USD
- Sumber trafik yang terbesar adalah hasil pencarian individual.
- Sumber trafik tidak memiliki hubungan dengan loyalitas user The Look.

## Poin Pertanyaan

1. Apakah problem statement yang kamu definisikan di awal dapat terukur ketercapaiannya? berikan pendapatmu!

- Menurut saya berdasarkan problem statement yang telah saya nyatakan di awal masih dapat dicapai secara terukur. <br>
Karena yang terlibat dalam pencapaian tujuan tersebut adalah beberapa bidang kerja di The Look, seperti UI/UX Designer, Software Engineer/Full Stack Developer yang akan bergerak di sisi aplikasi atau website The Look. Digital marketing, marketing, sales untuk melakukan riset dan juga engagement terhadap para user The Look.<br>
Kemudian apa yang ingin dicapai juga selaras dengan rumusan masalah, yaitu menjaga para user untuk tetap berada pada ekosistem The Look itu sendiri.
Dengan mencapai target - target tersebut, maka problem statement yang telah dinyatakan diawal dapat teratasi.<br>
Lalu, seberapa banyak jumlah user dengan retensi tinggi harus ditingkatkan? Karena sekarang hanya sebanyak 0.588% maka bisa ditingkatkan ke sekitar 2%-3% dengan melakukan engagement kepada mereka yang setidaknya sudah melakukan pemesanan dengan menggunakan The Loop sebanyak 2 kali pemesanan. Misalnya dengan memberikan promo khusus.<br>
Kemudian bagaimana cara kita mengetahui bahwa goal atau tujuan kita tercapai? Kita dapat melakukan pemerisakaan kemabli terhadap jumlah customer dengan retensi tinggi pada bulan Agustus atau Desember mendatang.<br><br>
2. Apa yang dimaksud dengan customer/user retention?
- Customer retention adalah berbagai tindakan yang dilakukan perusahaan untuk mempertahankan customer supaya tetap menggunakan produk atau layanan mereka.<br><br>


3. Jika platform The Look membuat fitur tambahan, apakah kamu akan merekomendasikan/memprioritaskan kepada user loyal? apa pertimbanganmu?
- Mempertahankan customer lama itu membutuhkan waktu yang lebih cepat serta efektif. Lebih mudah menjual kepada customer yang sudah mengenal brand Anda, daripada menghabiskan waktu untuk marketing sambil mencari customer baru. Sebanyak 25% - 40% dari total pendapatan bisnis yang sudah stabil, didapat dari returning customer. Repeat customer bisa melipatgandakan pendapatan hingga 3-7 kali lebih banyak daripada one-time buyer.